In [1]:
############################################################################
# 엘라스틱서치 8.x.x 버전일때 테스트 예시.
#
#===========================================================================
# [api_key 생성 방법]
# 출처 : https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/getting-started-python.html
# => kibana 접속 : localhost:5601->메뉴에서 Search>elasticsearch 클릭
# ->화면중간에 API KEY 생성 클릭> 이후 Name 입력하고, Contol security privileges(보안접근권한) 설정후 [Create API Key] 입력
# => python 코드에서는 api_key 인자로 encoded 값을 입력해야 함.
#{
#  "id": "1Rtxxxxxxxxxxxxxxxxx",
#  "name": "apikey",
#  "api_key": "3Pfxxxxxxxxxxxxxx",
#  "encoded": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx===",
#  "beats_logstash_format": "1Rtoxxxxxxxxxxxxxxxxx"
#}

# scikit-extra 설치
# !pip install scikit-learn-extra

# elasticsearch 버전 보기 
# !pip show elasticsearch

# pip로 elasticsearch 8.14.3으로 업데이트 
# !pip install --upgrade elasticsearch
############################################################################
import torch
import time
import os
import numpy as np
import sys

sys.path.append('../')
from utils import MyUtils
from es_8 import My_ElasticSearch

# settings.yaml 설정값 불러옴.
myutils = MyUtils(yam_file_path='../data/es_settings.yaml')
settings = myutils.get_options()

# elastic search 설정
es_url=settings['ES_URL']
es_index_name=settings['ES_INDEX_NAME']
es_index_file_path=settings['ES_INDEX_FILE_PATH']
es_api_key=settings['ES_API_KEY']
myes = My_ElasticSearch(es_url=es_url, index_file_path=es_index_file_path, api_key=es_api_key)
print(f'*ES정보: {myes}')

*ES정보: <es_8.main.My_ElasticSearch object at 0x7f1b952fec40>


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/elasticsearch/connection/http_urllib3.py:209: UserWarning: Connecting to https://10.10.4.10:9201 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 로딩
# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"
print(f'*임베딩 모델 {settings["EMBEDDING_MODEL"]} 로딩 시작==>')

# model_name = 없으면: 기본 'sentence-transformers/all-mpnet-base-v2' 모델임.
#embeddings = HuggingFaceEmbeddings()  # HuggingFace 임베딩을 생성합니다.
embedding = HuggingFaceEmbeddings(
    model_name=settings["EMBEDDING_MODEL"], 
    show_progress=True
)
print(f'*임베딩 모델: {embedding.model_name}')

# 임베딩 테스트
docs = ['제주도는 대한민국이 남쪽에 있는 섬이다.']
docs_vectors = embedding.embed_documents(docs)
print(docs_vectors)

*임베딩 모델 bongsoo/kpf-sbert-128d-v1 로딩 시작==>


/tmp/ipykernel_147257/587561479.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limit

*임베딩 모델: bongsoo/kpf-sbert-128d-v1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[-0.1490500420331955, -0.13548478484153748, 0.06716418266296387, 0.09104267507791519, 0.019638732075691223, -0.06333298236131668, 0.09045691043138504, 0.05989188700914383, -0.11224446445703506, -0.12992143630981445, -0.13975737988948822, 0.12996476888656616, -0.07497131079435349, -0.008917364291846752, -0.30811938643455505, 0.026844028383493423, -0.1107262372970581, -0.117447130382061, 0.09462910145521164, 0.17226745188236237, -0.03926313668489456, 0.18109464645385742, -0.01499598566442728, 0.04120016470551491, -0.03429461643099785, -0.3762201964855194, -0.2587670087814331, -0.36710312962532043, -0.16324147582054138, 0.1753007024526596, -0.18036913871765137, 0.009004500694572926, 0.5081278681755066, -0.17946484684944153, 0.2888895869255066, -0.1981419026851654, 0.04371872544288635, -0.11642713099718094, -0.12993785738945007, 0.0909293070435524, 0.03948690742254257, 0.12344682216644287, -0.05224553868174553, 0.11220505833625793, -0.016199031844735146, 0.06210748478770256, 0.00319715542

In [3]:
# 1.인덱스 생성
res=myes.create_index(index_name=es_index_name, delete=True)
print(f'rs:{res}')

*[create_index] self.es.indices.delete
*[create_index] self.es.indices.create
rs:{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mpower10u_128d_1_keword_test'}


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
# 2.인덱스에 데이터 추가
file_name = "제주도"
file_text = "제주도는 대한민국이 남쪽에 있는 섬이다."

# 벡터 생성
doc = ['제주도는 대한민국이 남쪽에 있는 섬이다.']
vector_list = embedding.embed_documents(docs)

# *vector0 에는 리스트 그대로 담아야 함. f'{vector_list[0]}' 식으로 넣으면 text로 들어가므로 오류남.
doc1 = {'rfile_name':f'{file_name}', 'rfile_text':f'{file_text}', 'vector0': vector_list[0]}

res=myes.insert(index_name=es_index_name, doc=doc1)
print(f'rs:{res}')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

*doc:{'rfile_name': '제주도', 'rfile_text': '제주도는 대한민국이 남쪽에 있는 섬이다.', 'vector0': [-0.1490500420331955, -0.13548478484153748, 0.06716418266296387, 0.09104267507791519, 0.019638732075691223, -0.06333298236131668, 0.09045691043138504, 0.05989188700914383, -0.11224446445703506, -0.12992143630981445, -0.13975737988948822, 0.12996476888656616, -0.07497131079435349, -0.008917364291846752, -0.30811938643455505, 0.026844028383493423, -0.1107262372970581, -0.117447130382061, 0.09462910145521164, 0.17226745188236237, -0.03926313668489456, 0.18109464645385742, -0.01499598566442728, 0.04120016470551491, -0.03429461643099785, -0.3762201964855194, -0.2587670087814331, -0.36710312962532043, -0.16324147582054138, 0.1753007024526596, -0.18036913871765137, 0.009004500694572926, 0.5081278681755066, -0.17946484684944153, 0.2888895869255066, -0.1981419026851654, 0.04371872544288635, -0.11642713099718094, -0.12993785738945007, 0.0909293070435524, 0.03948690742254257, 0.12344682216644287, -0.05224553868174553, 0

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
# 한꺼번에 인덱싱
es_index_name=settings['ES_INDEX_NAME']
op_data = {"index": {"_index": es_index_name}}
print(f"*op_data:{op_data}")

docs:list = []

file_name_list = ['한라산', '서귀포', '제주시', '성산읍']

file_text_list = ['한라산은 제주도에 있는 휴화산이다.', 
                 '대한민국 제주도 최남단에 있는 도시이다.',
                 '제주도의 행정/경제 중심 도시이다.',
                 '제주도 동쪽에 있는 읍으로, 성산일출봉과 제2공항 후보지로 유명하다.']

# vector는 한번에 구함.
vector_list = embedding.embed_documents(file_text_list)

# 인덱싱 doc 구성.
for file_name, file_text, vector in zip(file_name_list, file_text_list, vector_list):
   
    doc = {'rfile_name': file_name, 'rfile_text': file_text, 'vector0': vector}
    docs.append(op_data)
    docs.append(doc)
        
print(f"*docs:{docs}")

res=myes.RAG_bulk(docs=docs)
print(f'rs:{res}')

*op_data:{'index': {'_index': 'mpower10u_128d_1_keword_test'}}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

*docs:[{'index': {'_index': 'mpower10u_128d_1_keword_test'}}, {'rfile_name': '한라산', 'rfile_text': '한라산은 제주도에 있는 휴화산이다.', 'vector0': [-0.23051650822162628, -0.055954400449991226, 0.17652785778045654, 0.10361403971910477, -0.08508777618408203, 0.07885684818029404, 0.3802871108055115, -0.26859867572784424, -0.08067595213651657, 0.2670346200466156, -0.1282011866569519, 0.02894274890422821, -0.23442670702934265, 0.3000251054763794, -0.17227494716644287, -0.19574494659900665, -0.144011452794075, -0.33627912402153015, 0.5146533250808716, -0.26289066672325134, -0.047792255878448486, -0.03934041038155556, -0.2859087288379669, 0.1776248663663864, -0.13889232277870178, -0.26382580399513245, 0.24777460098266602, -0.21590013802051544, -0.0497458316385746, -0.10776741057634354, -0.24450410902500153, 0.08479783684015274, 0.4591793119907379, -0.06378626823425293, 0.04387318715453148, 0.07370118051767349, 0.3058598041534424, 0.20836085081100464, -0.16442561149597168, -0.17585641145706177, -0.0435565486

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
# BM25 검색
query="제주도"
min_score=0.1
res=myes.BM25_search(index_name=es_index_name, query=query, min_score=min_score)
print(f'rs:{res}')

file_name:제주도] score:0.40618476

file_name:성산읍] score:0.33081025

file_name:제주시] score:0.28145537

rs:[{'rfile_name': '제주도', 'rfile_text': '제주도는 대한민국이 남쪽에 있는 섬이다.', 'score': 0.40618476}, {'rfile_name': '성산읍', 'rfile_text': '제주도 동쪽에 있는 읍으로, 성산일출봉과 제2공항 후보지로 유명하다.', 'score': 0.33081025}, {'rfile_name': '제주시', 'rfile_text': '제주도의 행정/경제 중심 도시이다.', 'score': 0.28145537}]


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [20]:
from es_8 import  make_embedding_query_script

# 벡터 생성 후 임베딩 search
doc = ['대한민국 최남단에 있는 섬은 어디인가요?']
vector_list = embedding.embed_documents(doc)
script_query = make_embedding_query_script(qr_vector=vector_list[0], uid_list=None) # 쿼리를 만듬.   

embed_docs = myes.search_docs(index_name=es_index_name,
                              script_query=script_query,
                              k=3,
                              min_score=0.5)
'''
# 임베딩 search
embed_docs = myes.Embedding_search(index_name=es_index_name,
                                   huggingfaceembeddings=embedding,
                                   query = '대한민국 최남단에 있는 섬은 어디인가요?', 
                                   k=3, 
                                   min_score=0.5) 
'''
print(embed_docs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

*[search_docs] score:1.652169
rfile_name:서귀포

*[search_docs] score:1.6118428
rfile_name:제주도

*[search_docs] score:1.3053708
rfile_name:제주시

[{'rfile_name': '서귀포', 'score': 1.652169}, {'rfile_name': '제주도', 'score': 1.6118428}, {'rfile_name': '제주시', 'score': 1.3053708}]


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
# 모든 데이터 불러옴 
script_query = {"match_all": {}}
res=myes.search_docs(index_name=es_index_name, script_query=script_query)
print(f'rs:{res}')

*[search_docs] score:1.0
rfile_name:한라산

*[search_docs] score:1.0
rfile_name:서귀포

*[search_docs] score:1.0
rfile_name:제주시

*[search_docs] score:1.0
rfile_name:성산읍

*[search_docs] score:1.0
rfile_name:제주도

rs:[{'rfile_name': '한라산', 'score': 1.0}, {'rfile_name': '서귀포', 'score': 1.0}, {'rfile_name': '제주시', 'score': 1.0}, {'rfile_name': '성산읍', 'score': 1.0}, {'rfile_name': '제주도', 'score': 1.0}]


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [24]:
# 인덱스내 데이터 불러옴
data = {'rfile_name': '제주도'}
res=myes.search(index_name=es_index_name, data=data)
print(f'rs:{res}')

rs:{'took': 1, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.6931471, 'hits': [{'_index': 'mpower10u_128d_1_keword_test', '_id': '84-h-JQBI9PlGwx96QaJ', '_score': 0.6931471, '_source': {'rfile_name': '제주도', 'rfile_text': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 70만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.', 'vector0': [-0.1490500420331955, -0.13548478484153748, 0.06716418266296387, 0.09104267507791519, 0.019638732075691223, -0.06333298236131668, 0.09045691043138504, 0.05989188700914383, -0.11224446445703506, -0.12992143630981445, -0.13975737988948822, 0.12996476888656616, -0.07497131079435349, -0.008917364291846752, -0.30811938643455505, 0.026844028383493423, -0.1107262372970581, -0.117447130382061, 0.09462910145521164, 0.17226745188236237, -0.03926313668489456, 0.18109464645385742, -0.01499598566442728, 0.04120016470551491, -0.03429461643099785, -0.3762201964855194, -0.2587670087814331, -0.367103

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [26]:
# 인덱스에 데이터 업데이트
# =>업데이트할 _id, _type을 알아야함

# 쿼리 검색
data = {'rfile_name': '제주도'}
res=myes.search(index_name=es_index_name, data=data)
print(f'rs:{res}')
print('\n')

id_list = []
# 업데이트할 id를 구함
for hits in res['hits']['hits']:
    id=hits['_id']
    #print(f'id:{id}')
    
    id_list.append(id)
    
# 업데이트
for id in id_list:
    
    print(f'id:{id}')
    doc = {'rfile_text': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 70만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}
    
    res=myes.update(index_name=es_index_name, doc=doc, esid=id)
    print(f'res:{res}')
    print('\n')

rs:{'took': 0, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}




/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [25]:
# 인덱스내 데이터 삭제
data = {'rfile_name': '제주도'}
res=myes.delete(index_name=es_index_name, doc=data)
print(f'rs:{res}')

rs:{'took': 43, 'timed_out': False, 'total': 1, 'deleted': 1, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [27]:
# 인덱스 삭제
res=myes.delete_index(index_name=es_index_name)
print(f'rs:{res}')

rs:{'acknowledged': True}


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.10.4.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
